In [92]:
import os                             #### FOR MULTIPLE FEATURES X, MULTIPLE-in-calculation Y ####
import tensorflow as tf

W = tf.Variable(tf.zeros([4, 3]), name="weights")
b = tf.Variable(tf.zeros([3]), name="bias")

def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.join(os.getcwd(), file_name)])
    #filename_queue=tf.train.string_input_producer(['C:\\Users\\Thuy Nguyen\\Desktop\\Tensorflow learning\\iris.data'])
    
    reader = tf.TextLineReader(skip_header_lines=0) # there is no header to skip
    key, value = reader.read(filename_queue) # load the rest of the file to (key, value)
    
    # convert a Tensor from type string (the text line) in a tuple of tensor columns with the specified defaults,
    # which also sets the data type for each column
    decoded = tf.decode_csv(value, record_defaults=record_defaults) 
    
    # load 'batch_size' rows in a single tensor
    return tf.train.shuffle_batch(decoded, batch_size=batch_size, capacity=batch_size*50, min_after_dequeue=batch_size)
    #return decoded

    
# ------- define the training loop operations -------

def combine_inputs(X):
    return tf.matmul(X, W) + b

def inference(X):
    return tf.nn.softmax(combine_inputs(X))

def loss(X, Y):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(combine_inputs(X), Y))
        # REF: http://stackoverflow.com/questions/34240703/difference-between-tensorflow-tf-nn-softmax-and-tf-nn-softmax-cross-entropy-with
        # The softmax "squishes" the inputs so that sum(input) = 1; it's a way of normalizing. Shape of output of a
        # softmax is the same as the input, it just normalizes the values. Outputs of softmax can be interpreted as 
        # probabilities.
        # Ex:   a = tf.constant(np.array([[.1, .3, .5, .9]]))
        #       print s.run(tf.nn.softmax(a))
        #       [[ 0.16838508  0.205666    0.25120102  0.37474789]]
        # In contrast, tf.nn.softmax_cross_entropy_with_logits computes the cross entropy of the result after applying 
        # the softmax function (but it does it all together in a more mathematically careful way). It's similar to the 
        # result of:
        #       sm = tf.nn.softmax(x)
        #       ce = cross_entropy(sm)
        # Cross entropy is a summary metric-it sums across elements. Output of tf.nn.softmax_cross_entropy_with_logits 
        # on a shape [2,5] tensor is of shape [2,1] (the 1st dimension is treated as batch).
        # If you want to do optimization to minimize the cross entropy, AND you're softmaxing after your last layer, 
        # you should use tf.nn.softmax_cross_entropy_with_logits instead of doing it yourself, because it covers 
        # numerically unstable corner cases in the mathematically right way.
        
def inputs():
    sepal_length, sepal_width, petal_length, petal_width, label = \
        read_csv(10, "iris.data", [[0.0], [0.0], [0.0], [0.0], [""]])
    
    # pack all features in a single matrix, then transpose to have a matrix of 1 sample per row & 1 feature per column
    features = tf.transpose(tf.pack([sepal_length, sepal_width, petal_length, petal_width]))
    
    # convert class names to a 0 based class index
    is_setosa = tf.equal(label, ["Iris-setosa"])
    is_versicolor = tf.equal(label, ["Iris-versicolor"])
    is_virginica = tf.equal(label, ["Iris-virginica"])
    pack_label = tf.pack([is_setosa, is_versicolor, is_virginica]) # pack([x,y]) => [[1, 4],[2, 5]] #Pack along 1st dim
    label_number = tf.to_int32(tf.argmax(tf.to_int32(pack_label), 0))
                                            # argmax: return the index with the largest value across axes of a tensor
    return features, label_number

def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

def evaluate(sess, X, Y):
    Y_predicted = tf.cast(tf.argmax(inference(X), 1), tf.int32)
    print(sess.run(tf.reduce_mean(tf.cast(tf.equal(Y_predicted, Y), tf.float32))))
    
# ---------------------------------------------------


with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    # X has 4 features of [sepal_length, sepal_width, petal_length, petal_width]
    # Y is training_output of [label], it has 0 ("Iris-setosa"), 1 ("Iris-versicolor"), and 2 ("Iris-virginica") values
    X, Y = inputs()
    #print(sess.run(X)) # USE sess TO PRINT VALUES TO SCREEN
    
    total_loss = loss(X, Y)
    train_op = train(total_loss)    
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
 
    # actual training loop
    training_steps = 5
    for step in range(training_steps):
        sess.run([train_op])
    #    if step % 1 == 0: # for debug purpose, see how the loss gets decremented thru training steps
        print("loss: ", sess.run([total_loss]))
    evaluate(sess, X, Y)
    
    coord.request_stop()
    coord.join(threads)
    sess.close()

loss:  [0.84940404]
loss:  [0.66916871]
loss:  [1.2828152]
loss:  [0.95916575]
loss:  [1.3469915]
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Expect 5 fields but have 0 in record 0
	 [[Node: DecodeCSV_5 = DecodeCSV[OUT_TYPE=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_STRING], field_delim=",", _device="/job:localhost/replica:0/task:0/cpu:0"](ReaderRead_5:1, DecodeCSV_5/record_defaults_0, DecodeCSV_5/record_defaults_1, DecodeCSV_5/record_defaults_2, DecodeCSV_5/record_defaults_3, DecodeCSV_5/record_defaults_4)]]

Caused by op 'DecodeCSV_5', defined at:
  File "c:\users\thuy nguyen\anaconda3\envs\tensorflow\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\thuy nguyen\anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\thuy nguyen\anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
  

InvalidArgumentError: Expect 5 fields but have 0 in record 0
	 [[Node: DecodeCSV_5 = DecodeCSV[OUT_TYPE=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_STRING], field_delim=",", _device="/job:localhost/replica:0/task:0/cpu:0"](ReaderRead_5:1, DecodeCSV_5/record_defaults_0, DecodeCSV_5/record_defaults_1, DecodeCSV_5/record_defaults_2, DecodeCSV_5/record_defaults_3, DecodeCSV_5/record_defaults_4)]]

Caused by op 'DecodeCSV_5', defined at:
  File "c:\users\thuy nguyen\anaconda3\envs\tensorflow\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\thuy nguyen\anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\thuy nguyen\anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "c:\users\thuy nguyen\anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\user